In [1]:
def say(text):
    def _say(func):
        def __say(*args, **kwargs):
            print(text)
            return func(*args, **kwargs)
        return __say
    return _say

In [2]:
@say('Hello')
def add(a, b):
    return a + b

In [4]:
add(3, 4)

Hello


7

In [5]:
@say('Goodbye')
def add(a, b):
    return a + b

In [6]:
add(3, 4)

Goodbye


7

In [7]:
@say('Hello')
@say('Goodbye')
def add(a, b):
    return a + b

In [8]:
add(3, 4)

Hello
Goodbye


7

In [9]:
def add(a, b):
    return a + b

add = say('Hello')(add)

In [10]:
add(4, 5)

Hello


9

In [14]:
def add(a, b):
    return a + b

add = say('Goodbye')(add)
add = say('Hello')(add)

In [15]:
add(3, 4)

Hello
Goodbye


7

In [16]:
int('4')

4

In [17]:
type(int)

type

In [18]:
int.__call__('4')

4

In [19]:
add.__call__(3, 4)

Hello
Goodbye


7

In [20]:
class CallCounter:

    def __init__(self):
        self.counter = 0

    def __call__(self):
        self.counter += 1

In [21]:
my_func = CallCounter()

In [22]:
my_func.counter

0

In [23]:
my_func()

In [24]:
my_func.counter

1

In [25]:
my_func.__call__()

In [26]:
my_func.counter

2

In [27]:
from functools import wraps

In [29]:
class Say:

    def __init__(self, text):
        self.text = text

    def __call__(self, func):
        @wraps(func)
        def say(*args, **kwargs):
            print(self.text)
            return func(*args, **kwargs)
        return say

In [30]:
@Say('Hello')
def add(a, b):
    return a + b
    

In [31]:
add(3, 4)

Hello


7

In [32]:
%cd decorators/

/Users/mike/pya/Konferenzen/EuroPython/EuroPython2023/Tutorial/decorators


In [39]:
# %load argcheck.py
"""Check function arguments for given type.
"""

import functools


def check(*argtypes):
    """Function argument type checker.
    """

    def _check(func):
        """Takes the function.
        """

        @functools.wraps(func)
        def __check(*args):
            """Takes the arguments
            """
            if len(args) != len(argtypes):
                msg = f'Expected {len(argtypes)} but got {len(args)} arguments'
                raise TypeError(msg)
            for arg, argtype in zip(args, argtypes):
                if not isinstance(arg, argtype):
                    msg = f'Expected {argtypes} but got '
                    msg += f'{tuple(type(arg) for arg in args)}'
                    raise TypeError(msg)
            return func(*args)
        return __check
    return _check


In [34]:
type(1)

int

In [35]:
type(True)

bool

In [36]:
isinstance(True, int)

True

In [37]:
bool.mro()

[bool, int, object]

In [38]:
True + True

2

In [40]:
@check(float, float)
def add(a, b):
    return a + b

In [42]:
add(3., 4.)

7.0

In [44]:
# %load registering.py
"""A function registry.
"""

import functools

registry = {}


def register_at_call(name):
    """Register the decorated function at call time.
    """

    def _register(func):
        """Takes the function.
        """

        @functools.wraps(func)
        def __register(*args, **kwargs):
            """Takes the arguments.
            """
            registry.setdefault(name, []).append(func)
            return func(*args, **kwargs)
        return __register
    return _register


def register_at_def(name):
    """Register the decorated function at definition time.
    """

    def _register(func):
        """Takes the function.
        """
        registry.setdefault(name, []).append(func)

        return func
    return _register


In [45]:
@register_at_call('simple')
def add1(a, b):
    return a + b

In [46]:
registry

{}

In [47]:
add1(3, 4)

7

In [48]:
registry

{'simple': [<function __main__.add1(a, b)>]}

In [49]:
@register_at_call('simple')
def add2(a, b):
    return a + b

In [50]:
add2(10, 20)

30

In [51]:
registry

{'simple': [<function __main__.add1(a, b)>, <function __main__.add2(a, b)>]}

In [52]:
registry.clear()

In [53]:
registry

{}

In [54]:
@register_at_def('complex')
def add3(a, b):
    return a + b

In [55]:
registry

{'complex': [<function __main__.add3(a, b)>]}

In [56]:
add3(3, 4)

7

In [57]:
registry

{'complex': [<function __main__.add3(a, b)>]}

# Class decorator

In [58]:
def mark(cls):
    cls.new = 100
    return cls

In [59]:
@mark
class A:
    pass

In [60]:
A.new

100

In [61]:
class B(A):
    pass

In [62]:
B.new

100

In [63]:
class C:

    def meth1(self):
        pass

    def meth2(self):
        pass

In [64]:
C.__dict__

mappingproxy({'__module__': '__main__',
              'meth1': <function __main__.C.meth1(self)>,
              'meth2': <function __main__.C.meth2(self)>,
              '__dict__': <attribute '__dict__' of 'C' objects>,
              '__weakref__': <attribute '__weakref__' of 'C' objects>,
              '__doc__': None})

In [65]:
def required_methods(cls):
    for x in range(1, 6):
        if not f'meth{x}' in cls.__dict__:
            raise AttributeError(f'attribute meth{x} not found')
    return cls

In [66]:
@required_methods
class C:

    def meth1(self):
        pass

    def meth2(self):
        pass

AttributeError: attribute meth3 not found

In [69]:
def required_methods(n=5):
    def _required_methods(cls):
        for x in range(1, n+1):
            if not f'meth{x}' in cls.__dict__:
                raise AttributeError(f'attribute meth{x} not found')
        return cls
    return _required_methods

In [72]:
@required_methods(2)
class C:

    def meth1(self):
        pass

    def meth2(self):
        pass

In [73]:
class D(C):
    pass

In [74]:
def super_deco(deco):
    def _super_deco(cls):
        for name, obj in cls.__dict__.items():
            if callable(obj):
                setattr(cls, name, deco(obj))
        return cls
    return _super_deco

In [75]:
@super_deco(say('Hello'))
class All:

    def meth1(self):
        pass

    def meth2(self):
        pass 

In [76]:
a = All()

In [77]:
a.meth1()

Hello


In [78]:
def assert_fluid(cls):
    assert 0 <= cls.temperature <= 100
    return cls

In [79]:
@assert_fluid
class Water:
    temperature = 25

In [80]:
@assert_fluid
class Ice:
    temperature = -25

AssertionError: 

In [81]:
%ls

argcheck.py*          logged.py*            registering.py*
cached.py*            method_name_check.py*


In [83]:
# %load method_name_check.py
"""Class decorator to check method name length."""


def check_name_length(max_len=30):
    """Check method name length.

    Raises a `NameError` if one method name of a decoratoed class is
    longer than `max_len`.
    """
    def _check_name_length(cls):
        for name, obj in cls.__dict__.items():
            if callable(obj) and len(name) > max_len:
                msg = (f'name `{name}` too long,\n  ' + len('NameError') * ' ' +
                       f'found {len(name)} characters, only {max_len} are allowed')
                raise NameError(msg)
        return cls
    return _check_name_length

In [89]:
@check_name_length()
class X:

    def meth1(self):
        pass

    def my_method_that_has_a_very_decriptive_name(self):
        pass

NameError: name `my_method_that_has_a_very_decriptive_name` too long,
           found 41 characters, only 30 are allowed

In [86]:
Water.__dict__

mappingproxy({'__module__': '__main__',
              'temperature': 25,
              '__dict__': <attribute '__dict__' of 'Water' objects>,
              '__weakref__': <attribute '__weakref__' of 'Water' objects>,
              '__doc__': None})

In [122]:
MAX_LEN = 30

def check_name_length2(max_len=MAX_LEN):
    """Check method name length.

    Raises a `NameError` if one method name of a decoratoed class is
    longer than `max_len`.
    """
    def _check_name_length(cls):
        for name, obj in cls.__dict__.items():
            if callable(obj) and len(name) > max_len:
                msg = (f'name `{name}` too long,\n  ' + len('NameError') * ' ' +
                       f'found {len(name)} characters, only {max_len} are allowed')
                raise NameError(msg)
        return cls
    print(max_len)
    if type(max_len) is type:
        cls = max_len
        max_len = MAX_LEN
        return _check_name_length(cls)
    else:
        return _check_name_length



In [123]:
check_name_length2?

Signature: check_name_length2(max_len=30)
Docstring:
Check method name length.

Raises a `NameError` if one method name of a decoratoed class is
longer than `max_len`.
File:      /var/folders/p_/yckgs9wn0xbcmqc92zk3z4240000gp/T/ipykernel_85844/143033117.py
Type:      function

In [120]:
@check_name_length2
class X:

    def meth1(self):
        pass

    def my_method_that_has_a_very_decriptive_name(self):
        pass

<class '__main__.X'>


NameError: name `my_method_that_has_a_very_decriptive_name` too long,
           found 41 characters, only 30 are allowed

In [130]:
from timeit import default_timer

def measure_time(repeat=1):
    def _measure_time(func):
        times = []
        @wraps(func)
        def __measure_time(*args, **kwargs):
            start = default_timer()
            for _ in range(repeat):
                res = func(*args, **kwargs)
            duration = (default_timer() - start) / repeat
            times.append(duration)
            return res
        __measure_time.times = times
        return __measure_time
    return _measure_time

In [131]:
@measure_time()
def add(a, b):
    return a + b

In [132]:
add(3, 4)
add.times

[5.207955837249756e-06]

In [137]:
@measure_time(100_000)
def add(a, b):
    return a + b
add(3, 4)
add.times

[1.7689833184704185e-07]

In [148]:
for n in range(1, 9):
    repeat = 10 ** n
    @measure_time(repeat)
    def add(a, b):
        return a + b
    add(3, 4)
    print(f'{repeat:12d} {add.times[0]:.2e}')

          10 3.50e-07
         100 1.73e-07
        1000 1.65e-07
       10000 1.67e-07
      100000 1.81e-07
     1000000 6.32e-08
    10000000 5.15e-08
   100000000 5.00e-08
